# Moving to the Cloud

## Setup TensorFlow locally

Follow the instructions to install TensorFlow locally on your computer:

https://www.tensorflow.org/install/

If you install Python, make sure to install a version that is supported (e.g. as of 1/26/19, Python 3.7 is not suppoerted!).

You can also skip this section and proceed with "Create VM" if you get stuck with the installation.

Test your installation by copying the following cell into a file and executing it on your computer:

In [ ]:
import tensorflow as tf
print('TensorFlow version {}, gpu_available={}, built_with_cuda={}'.format(
    tf.__version__, tf.test.is_gpu_available(), tf.test.is_built_with_cuda()))

## Create VM

**Prerequisite** : Before using any Cloud services, you will need to set up a billing account (https://console.cloud.google.com/billing) and register a credit card. Once your credit card is validated (by charing and immediately reimbursing a small amount), you will get 300 USD credit for one year.

Participants of the workshop will get a 50 USD voucher and don't need to set up any billing.

1. Create a new virtual machine (use **default settings**; see section "Find better parameters" for how to set up a VM with GPU) in Google Cloud Console:

    https://console.cloud.google.com/compute/instances

2. Once the VM has started (green icon), click on the "SSH" button in the "Connect" column - if you have copy'n'paste problems with the web terminal, try with another browser.

3. Install tensorflow

    `$ sudo apt-get -y install python3-pip python3-dev && pip3 install tensorflow`

4. Check the installation

    `$ python3 -c'import tensorflow as tf; print(tf.__version__)'`

5. Tip: If you turn down the VM when you don't need it, your credits will last a lot longer :-)

## Train as Python program

Below is a minimal program that trains a linear model using data from cloud. Copy it into a file and start it on your computer / cloud (don't forget to run the program with `python3` on Cloud if you used above commands).

In [ ]:
import tensorflow as tf

data_path = 'gs://amld-datasets/zoo_img'
batch_size = 100
train_steps = 500
eval_steps = 200
labels = [label.strip() for label in tf.gfile.GFile('{}/labels.txt'.format(data_path))]

feature_spec = {
    'label': tf.FixedLenFeature(shape=[1], dtype=tf.int64),
    'img_64': tf.FixedLenFeature(shape=[64, 64], dtype=tf.int64),
}

def parse_example(serialized_example):
  features = tf.parse_single_example(serialized_example, feature_spec)
  label = features.pop('label')
  label = tf.one_hot(tf.squeeze(label), len(labels))
  img_64 = tf.cast(features['img_64'], tf.float32) / 255.
  return img_64, label

ds_train = tf.data.TFRecordDataset(tf.gfile.Glob('{}/train-*'.format(data_path))).map(parse_example).batch(batch_size)
ds_test  = tf.data.TFRecordDataset(tf.gfile.Glob('{}/test-*'.format(data_path))).map(parse_example).batch(batch_size)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64,)),
    tf.keras.layers.Dense(len(labels), activation='softmax')
])
model.compile(
    optimizer=tf.train.AdamOptimizer(0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

history = model.fit(ds_train, steps_per_epoch=train_steps, epochs=1)
print('eval: ', model.evaluate(ds_test, steps=eval_steps))

model.save('linear.h5')

## Use your own data

1. Create a new storage bucket

    https://console.cloud.google.com/storage/browser

2. Upload data with commands below

3. Optionally set access permissions (e.g. give `allUsers` "Storage Object Viewer" access)

By the way : All workshop Colabs also work with paths that are inside cloud storage buckets!

In [ ]:
data_src_directory = '/content/gdrive/My Drive/amld_data/zoo_img'
# YOUR ACTION REQUIRED:
# Change the bucket name to the bucket that you created and have write access to.
data_dst_bucket = 'gs://amld-datasets'

In [ ]:
# YOUR ACTION REQUIRED:
# Authenticate Drive / Cloud (see "amld mount data" snippet)


In [ ]:
!gsutil cp -R "$data_src_directory" "$data_dst_bucket"

In [ ]:
# YOUR ACTION REQUIRED:
# Change the data_path in above training script to your own data and re-run.
# Note that you may have to update access rights accordingly.

# ----- Optional part -----

## Find better parameters

Extend the above program. Some suggestions:

- Specify parameters from command line using the `argparse` module.
- Make the model more complicated (like in [2_keras.ipynb](https://github.com/tensorflow/workshops/tree/master/extras/amld/notebooks/solutions/2_keras.ipynb)), also specify these as parameters.
- Store the evaluation results and training parameters to disk (e.g. as JSON).
- Write a script that explores different parameters.
- Write a script that shows a summary of results.
- Log results to TensorBoard (like in [3_eager.ipynb](https://github.com/tensorflow/workshops/tree/master/extras/amld/notebooks/solutions/3_eager.ipynb)).

**Pro Tip** : Want to use GPU and running into CUDA & TensorFlow library problems? You might find the following magic Cloud Shell command helpful (taken from [Cloud Docs](https://cloud.google.com/deep-learning-vm/docs/cli)):

    gcloud compute instances create gpu-1 --zone=us-east1-b --image-family=tf-latest-gpu --image-project=deeplearning-platform-release --maintenance-policy=TERMINATE --accelerator="type=nvidia-tesla-p100,count=1" --metadata="install-nvidia-driver=True"

This command will create a new VM instance called "gpu-1" and start it.

Note that you first need to apply for **GPU Quota**

https://console.cloud.google.com/iam-admin/quotas?metric=GPUs%20(all%20regions),NVIDIA%20P100%20GPUs

## ! Before you leave !

1. If you were at the Workshop, please give us your feedback !
2. You probably want to stop your VMs now, especially if you were running expensive hardware for fast training...